# Predspracovanie údajov

---

**Názov súboru:**  `predspracovanie.ipynb`

**Účel notebooku:**  Cieľom tohto súboru je dôsledné predspracovanie databázy pacientov zo súboru `databaza.xlsx`.  Obsahuje čistenie, extrakciu, transformáciu a prípravu údajov pre ďalšie modelovanie pomocou strojového učenia, konkrétne na predikciu výskytu a typu synkopy.

---
### 1. Načítanie a transpozícia údajov

Súbor `databaza.xlsx` je načítaný a transponovaný, aby riadky zodpovedali jednotlivým pacientom.

**Čistenie dát:**  Odstraňujú sa irelevantné a pomocné stĺpce ako napr. _"Navigácia"_, _"Záver"_ alebo rôzne poznámky.


In [1]:
# Načítanie základných knižníc na prácu s dátami
import pandas as pd
from datetime import datetime
import re
import numpy as np
import warnings
from collections import Counter
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
file_path = r"C:\Users\admin\PycharmProjects\pythonProject3\databaza.xlsx"
# Načítanie dát zo súboru do DataFrame
df = pd.read_excel(file_path, engine="openpyxl")
df_transposed = df.T
df_transposed.columns = df_transposed.iloc[0]
df_transposed = df_transposed[1:]
# Odstránenie vyšetrení a diagnostických stĺpcov
columns_to_drop = [
    "ECHO", "ECHOKG", "EMG", "EEG", "EKG", "EKG Holter", "Holter", "USG ECC",
    "CT mozgu", "MRI mozgu", "DG", "*DG:", "Záver ()"
]
df_transposed = df_transposed.drop(columns=[col for col in columns_to_drop if col in df_transposed.columns], errors='ignore')

df_transposed = df_transposed[~df_transposed.apply(lambda row: row.astype(str).str.contains("Poznámky", na=False)).any(axis=1)]

if "Navigácia" in df_transposed.columns:
    df_transposed.drop(columns=["Navigácia"], inplace=True)
df_transposed.reset_index(drop=True, inplace=True)

### 2. Spracovanie dátumov

Dátumy (napr. _"Dátum"_, _"Kedy bolo posledné odpadnutie?"_) sú konvertované na typ `datetime`.

**Oprava neplatných hodnôt:**  Pomocou funkcie `fix_invalid_dates()` sa nahrádzajú neplatné alebo príliš staré dátumy priemerným platným dátumom zo zvyšku dát.


In [2]:
# Zoznam stĺpcov, ktoré obsahujú dátumové informácie
date_columns = [
    "Dátum",
    "Začiatok ťažkostí (približný dátum)?",
    "Kedy bolo posledné odpadnutie?",
    "Kedy boli ťažkosti najhoršie?"
]
# Konverzia dátumových reťazcov na objekt typu datetime.date
for col in date_columns:
    if col in df_transposed.columns:
        df_transposed[col] = pd.to_datetime(df_transposed[col], errors="coerce").dt.date

if "Dátum" in df_transposed.columns:
    df_transposed["Dátum"] = pd.to_datetime(df_transposed["Dátum"], errors='coerce').dt.date 

if df_transposed["Dátum"].dtype == "object":
    ref_year = min(df_transposed["Dátum"].dropna()).year
else:
    ref_year = 2000
# Funkcia nahrádza chýbajúce alebo neplatné dátumy priemerom platných dátumov
def fix_invalid_dates(df):
    df["Dátum"] = pd.to_datetime(df["Dátum"], errors='coerce').dt.date
    valid_dates = df["Dátum"].dropna()
    mean_timestamp = pd.to_datetime(valid_dates).mean()
    mean_date = mean_timestamp.date()
    df["Dátum"] = df["Dátum"].apply(lambda x: mean_date if pd.isna(x) or x.year < 2017 else x)
    return df
df_transposed = fix_invalid_dates(df_transposed)


### 3. Extrakcia dátumu narodenia a pohlavia
Z údajov o rodnom čísle sa dekóduje dátum narodenia a pohlavie.

**Pravidlá dekódovania:**  
- Pre ženy: mesiac > 50  
- Podporované sú 10-miestne aj 6-miestne rodné čísla

**Použitá funkcia:** 
```python 
def extract_birthdate_and_gender(rodne_cislo)


In [3]:
# Funkcia na extrakciu dátumu narodenia a pohlavia z rodného čísla
def extract_birthdate_and_gender(rodne_cislo):
    if pd.isna(rodne_cislo):
        return np.nan, np.nan
    rodne_cislo = str(rodne_cislo).replace("/", "").strip()  

    if not re.fullmatch(r'\d{6,10}', rodne_cislo): 
        return np.nan, np.nan
    try:
        year = int(rodne_cislo[:2])
        month = int(rodne_cislo[2:4])
        day = int(rodne_cislo[4:6])
        if len(rodne_cislo) == 10:
            serial_number = int(rodne_cislo[6:9])
            if serial_number < 500:
                year += 1900
            else:
                year += 2000
        else:
            year += 1900 if year >= 20 else 2000
        # Rozlíšenie pohlavia – ženy majú k mesiacu pripočítané 50
        if month > 50:
            gender = "F"
            month -= 50
        else:
            gender = "M"
        birth_date = datetime(year, month, day).date()
        return birth_date, gender
    except ValueError:
        return np.nan, np.nan

### 4. Výpočet dátumu narodenia, pohlavia a veku pacienta

V tomto kroku sa z údajov o rodnom čísle automaticky extrahuje dátum narodenia a pohlavie pacienta pomocou funkcie `extract_birthdate_and_gender()`.

**Výpočet veku:**  Vek pacienta sa počíta ako **rozdiel medzi rokom vyšetrenia (`Dátum`) a rokom narodenia (`Dátum narodenia`). Ak chýba aspoň jedna z týchto hodnôt, výsledkom je `NaN`.

**Čistenie vekových hodnôt:**  
Na zabezpečenie konzistentnosti sú neplatné alebo záporné veky nahradené hodnotou `-1`.


In [4]:
# Aplikácia extrakcie dátumu narodenia a pohlavia
if "Rodné číslo" in df_transposed.columns:
    df_transposed["Datum narodenia"], df_transposed["Pohlavie"] = zip(
        *df_transposed["Rodné číslo"].apply(lambda x: extract_birthdate_and_gender(str(x))))
# Výpočet veku ako rozdiel medzi dátumom vyšetrenia a dátumom narodenia
df_transposed["Vek"] = df_transposed.apply(
    lambda row: (row["Dátum"].year - row["Datum narodenia"].year) if pd.notna(row["Dátum"]) and pd.notna(row["Datum narodenia"]) else np.nan,
    axis=1
)
df_transposed["Vek"] = df_transposed["Vek"].apply(lambda x: -1 if pd.isna(x) or x < 0 else x)

### 5. Zlúčenie údajov o liekoch do jedného stĺpca (`Aké lieky užívate`)

Hodnoty z viacerých stĺpcov medzi `Aké lieky užívate` a `Dátum narodenia` sa skombinujú do jedného textového stĺpca s názvom `Aké lieky užívate`.



In [5]:
start_col = "Aké lieky užívate"
end_col = "Datum narodenia"

if start_col in df_transposed.columns and end_col in df_transposed.columns:
    start_idx = df_transposed.columns.get_loc(start_col)
    end_idx = df_transposed.columns.get_loc(end_col)
    lieky_columns = df_transposed.iloc[:, start_idx:end_idx]
    df_transposed["Aké lieky užívate"] = lieky_columns.apply(
        lambda row: ', '.join(row.dropna().astype(str).replace(["-1", "nan", "Názov"], "").str.strip()).strip(", "),
        axis=1
    )
    columns_to_drop = df_transposed.columns[start_idx+1:end_idx]
    df_transposed.drop(columns=columns_to_drop, inplace=True)
df_transposed["Aké lieky užívate"] = df_transposed["Aké lieky užívate"].replace("", "0")

### 6. Identifikácia prítomnosti synkopy a jej typu

**Cieľ:**  Z textového záveru HUT testu identifikovať:

- Binárnu premennú `Synkopa`  
  - `1` = pacient mal synkopu  
  - `0` = pacient synkopu nemal  

- Typ synkopy podľa klasifikácie VASIS:  
  - `I` – zmiešaná forma  
  - `IIa` – kardioinhibičná bez asystólie  
  - `IIb` – kardioinhibičná s asystóliou  
  - `III` – vazodepresorová

**Po identifikácii:**  
- Odstráni sa pôvodný textový stĺpec `Záver HUT`


In [6]:
# Určenie, či bol výskyt synkopy – ak je v texte HUT testu zmienka o „VASIS“, ide o pozitívny prípad
df_transposed["Synkopa"] = df_transposed["Záver HUT"].apply(lambda x: 1 if isinstance(x, str) and re.search(r"VASIS ", x, re.IGNORECASE) else 0)
# Funkcia na detekciu konkrétneho typu synkopy podľa textových vzorov
def get_syncope_type(text):
    if not isinstance(text, str):
        return -1
    text = text.upper()
    type_mapping = {
        r"VASIS I|1\b": "VASIS I",
        r"VASIS (IIA|2A)\b": "VASIS IIa",
        r"VASIS (IIB|2B|ILB|IIB)\b": "VASIS IIb",
        r"VASIS (III|3)\b": "VASIS III",
    }
    for pattern, syncope_type in type_mapping.items():
        if re.search(pattern, text):
            return syncope_type
    return -1

df_transposed["Typ Synkopy"] = df_transposed.apply(lambda row: get_syncope_type(row["Záver HUT"]) if row["Synkopa"] == 1 else "NO CLASS", axis=1)
df_transposed = df_transposed.drop(columns=["Záver HUT"], errors='ignore')
hut_test_column = "HUT Test"


### 7. Štruktúrovanie stĺpcov a čistenie hodnôt

**Cieľ:**

- **Premiestniť** dôležité stĺpce (_napr. `Vek`, `Pohlavie`, `Synkopa` atď._) hneď za stĺpec `Dátum`
- **Odstrániť** pôvodný stĺpec `Rodné číslo`
- **Štandardizovať** textové odpovede (_napr. `áno`, `nie`_) a previesť ich na číselné hodnoty:
  - `áno` → `1`  
  - `nie` → `0`  
  - nevyplnené alebo nejasné → `-1`
- **Zabezpečiť**, že všetky relevantné hodnoty sú číselného formátu (napr. `Iné`)
- Konverzia všetkých číselných hodnôt na typ Int64 pre jednotnú prácu s `NaN`
- **Overenie**, či sa v dátach nenachádzajú duplikované názvy stĺpcov  
  (_napr. po importoch z viacerých verzií dotazníkov_)


In [7]:
# Zistenie pozícií dôležitých stĺpcov
datum_index = df_transposed.columns.get_loc("Dátum")
hut_test_index = df_transposed.columns.get_loc(hut_test_column)
# Dôležité stĺpce, ktoré chceme zoskupiť hneď za dátum
columns_to_move = ["Datum narodenia", "Pohlavie", "Vek", "Synkopa", "Typ Synkopy"]
existing_columns_to_move = [col for col in columns_to_move if col in df_transposed.columns]
# Nové poradie stĺpcov – najskôr pôvodné, potom tie dôležité, potom zvyšok
new_column_order = (
    df_transposed.columns[:datum_index + 1].tolist() +
    existing_columns_to_move +
    [col for col in df_transposed.columns[datum_index + 1:] if col not in existing_columns_to_move]
)

df_transposed = df_transposed[new_column_order]
df_transposed = df_transposed.drop(columns=["Rodné číslo"], errors='ignore')
# Funkcia na vyčistenie a zjednotenie odpovedí typu text (áno/nie/neuvedené)
def clean_text(value):
    if isinstance(value, str):
        value = value.strip().lower()
        value = value.replace("nienie", "nie")
        value = value.replace("áno", "ano")
        value = value.replace("niw", "nie")
        value = value.strip().upper()
    return value
df_transposed = df_transposed.map(clean_text)
# Mapovanie textových odpovedí na číselné reprezentácie
translation_rules = {
    'ANO': 1,
    'NIE': 0,
    'X': 1,
    'NEUVEDENE': -1,
    'POZIT': 1,
    'NEGAT': 0,
    'POZIT.':1,
    'NEGAT.': 0,
    'NEUVEDENÉ': -1,
    '': -1
}
df_transposed = df_transposed.replace(translation_rules)
# Funkcia, ktorá nahrádza nečíselné hodnoty v danom stĺpci číslom 1
def replace_non_numeric_with_one(df: pd.DataFrame, column_name: str) -> None:
    df[column_name] = df[column_name].apply(lambda x: 1 if not str(x).replace('.', '', 1).isdigit() else x)

replace_non_numeric_with_one(df_transposed, 'Iné')
# Vyplnenie všetkých zostávajúcich NaN hodnot -1
df_transposed.fillna(-1, inplace=True)
# Funkcia prevedie všetky stĺpce s typom float na typ Int64 (ktorý podporuje NaN)
def convert_floats_to_int_inplace(df: pd.DataFrame) -> None:
    for col in df.columns:
        if pd.api.types.is_float_dtype(df[col]):
            df[col] = df[col].astype('Int64')
convert_floats_to_int_inplace(df_transposed)



### 8. Prepis a spracovanie blokových otázok (A–R)

**Cieľ**:
- Zabezpečiť jednoznačnosť názvov stĺpcov (ak sa niektoré opakujú),
- Premenovať otázky na kódy A1–A10, B1–B5, ... pre zjednodušenú orientáciu,
- Vytvoriť reprezentáciu prítomnosti odpovede v jednotlivých blokoch:
  - ak aspoň jedna odpoveď v bloku (napr. A) je `1`, nastaví sa premenná `A = 1`,
- V prípade, že všetky odpovede v bloku sú `-1`, nastaví sa aj bloková premenná na `-1`.


In [8]:
# Zistenie opakujúcich sa stĺpcov a ich premenovanie (napr. „Búšenie srdca“, „Nepamätám sa“ sa môžu opakovať)
column_counts = Counter(df_transposed.columns)
new_columns = []
seen = {}
for col in df_transposed.columns:
    if column_counts[col] > 1:
        if col in seen:
            seen[col] += 1
        else:
            seen[col] = 1
        new_col_name = f"{col}_{seen[col]}"
    else:
        new_col_name = col
    new_columns.append(new_col_name)
df_transposed.columns = new_columns
# Prehľadné skracovanie dlhých otázok do blokových označení
df_transposed.rename(columns={
    'HUT Test': 'A',
    'Vykonaný áno/nie': 'A1',
    'Vstupný tlak': 'A2',
    'Vstupný pulz': 'A3',
    'Tlak po sklopení': 'A4',
    'Pulz po sklopení': 'A5',
    'Tlak po podaní NTG': 'A6',
    'Pulz po podaní NTG': 'A7',
    'Tlak po ukončení (po sklopení)': 'A8',
    'Pulz po ukončení (po sklopení)': 'A9',
    'Výsledok Testu': 'A10',
    'Som vyšetrovaný pre': 'B',
    'Stratu vedomia': 'B1',
    'Pocity hroziacej straty vedomia': 'B2',
    'Stav po resuscitácii': 'B3',
    'Stav po eplileptickom záchvate': 'B4',
    'Opakované pády': 'B5',
    'Moje ťažkosti': 'C',
    'Začiatok ťažkostí (približný dátum)?': 'C1',
    'Počet odpadnutí spolu?': 'C2',
    'Kedy bolo posledné odpadnutie?': 'C3',
    'Kedy boli ťažkosti najhoršie?': 'C4',
    'Popíšte ťažkosti / V akej situácii vznikli': 'D',
    'Strata vedomia pri státí?': 'D1',
    'Strata vedomia do 1 minúty po postavení sa?': 'D2',
    'Pri chôdzi?': 'D3',
    'Pri fyzickej námahe (akej?)': 'D4',
    'Strata vedomia v sede?': 'D5',
    'Strata vedomia poležačky?': 'D6',
    'Čo viedlo k strate vedomia': 'E',
    'Preľudnené priestory?': 'E1',
    'Dusné prostredie?': 'E2',
    'Teplé prostredie?': 'E3',
    'Pohľad na krv?': 'E4',
    'Nepríjemné emócie (strach, úzkosť, rozrušenie, odpor, pohľad na násilie)': 'E5',
    'Medicínsky výkon?': 'E6',
    'Bolesť?': 'E7',
    'Dehydratácia?': 'E8',
    'Menštruácia?': 'E9',
    'Strata krvi?': 'E10',
    'Vznikla strata vedomia pri niektorej z týchto situácií?': 'F',
    'Pri stolici': 'F1',
    'Pri močení': 'F2',
    'Pri kašli': 'F3',
    'Pri kýchaní/smrkaní nosa': 'F4',
    'Pri jedení/prehĺtaní': 'F5',
    'Po náhlej bolesti': 'F6',
    'Počas fyzickej námahy': 'F7',
    'Pri hlade': 'F8',
    'Pri nedostatku spánku, únave': 'F9',
    'Iné1': 'F10',
    'Užili ste hodinu pred stratou vedomia nejaké lieky alebo alkohol?': 'G',
    'Čo ste cítili tesne pred stratou vedomia?': 'H',
    'Pocit na zvracanie alebo zvracanie': 'H1',
    'Pocit tepla/horúco': 'H2',
    'Pot': 'H3',
    'Zahmlievanie pred očami': 'H4',
    'Hučanie v ušiach': 'H5',
    'Búšenie srdca1': 'H6',
    'Búšenie srdca_2': 'H7',
    'Bolesť na hrudníku': 'H8',
    'Neobvyklý zápach': 'H9',
    'Neobvyklé zvuky': 'H10',
    'Poruchy reči alebo slabosť polovice tela': 'H11',
    'Nepociťoval som nič zvláštne': 'H12',
    'Nepamätám sa1': 'H13',
    'Iné2': 'H14',
    'Ako dlho trvali tieto pocity pred stratou vedomia?': 'I',
    'Niekoľko sekúnd1': 'I1',
    'Do 1 miníty': 'I2',
    'Do 5 minút1': 'I3',
    'Viac ako 5 minút1': 'I4',
    'Čo ste urobili pri hroziacej strate vedomia?': 'J',
    'Sadol som si': 'J1',
    'Ľahol som si': 'J2',
    'Nestihol som urobiť nič pretože som stratil vedomie': 'J3',
    'Ak boli prítomní svedkovia, ako dlho podľa nich trvalo bezvedomie?': 'K',
    'Niekoľko sekúnd2': 'K1',
    'Do minúty': 'K2',
    'Do 5 minút2': 'K3',
    'Viac ako 5 minút2': 'K4',
    'Mali ste kŕče počas bezvedomia (v prípade svedkov udalosti)?': 'L',
    'Odišla vám stolica alebo moč počas bezvedomia?': 'M',
    'Pamätáte si na udalosti po strate vedomia?': 'N',
    'Mali ste pohryzený jazyk, pery?': 'N1',
    'Udreli ste sa pri páde, boli ste poranení v dôsledku pádu?': 'N2',
    'Po prebratí ste podľa údajov svedkov boli viac ako 30 minút dozerientovaní?': 'N3',
    'Bolela vás hlava alebo svaly?': 'N4',
    'Aj po prebratí ste pociťovali nevoľnosť?': 'N5',
    'Cítili ste sa normálne': 'N6',
    'Nepamätáte sa2': 'N7',
    'Výskyt ochorení vo vašej rodine': 'O',
    'Náhle úmrtie člena rodiny (v akom veku?)': 'O1',
    'Ochorenie srdca_1': 'O2',
    'Ochorenie srdca_2': 'O3',
    'Srdcová arytmia/kardiostimulátor': 'O4',
    'Ochoria mozgu/epilepsia': 'O5',
    'Na aké ochorenia ste sa doteraz liečili?': 'P',
    'Ochorenie srdca_3': 'P1',
    'Ochorenie srdca_4': 'P2',
    'Ochorenie chlopní': 'P3',
    'Srdcová slabosť': 'P4',
    'Koronárna chorova srdca': 'P5',
    'Srdcové arytmie': 'P6',
    'Búšenie srdca2': 'P7',
    'Búšenie srdca_4': 'P8',
    'Bolesti na hrudníku': 'P9',
    'Vysoký tlak krvi': 'P10',
    'Nízky tlak krvi': 'P11',
    'Závraty': 'P12',
    'Ochorenia obličiek': 'P13',
    'Diabetes (cukrovka)': 'P14',
    'Anémia': 'P15',
    'Astma': 'P16',
    'Ochorenia pľúc': 'P17',
    'Ochorenia priedušiek': 'P18',
    'Ochorenia žalúdka': 'P19',
    'Ochorenia čreva': 'P20',
    'Ochorenia štítnej žľazy': 'P21',
    'Endokrinologické ochorenia': 'P22',
    'Bolesti hlavy': 'P23',
    'Neurologické ochorenia': 'P24',
    'Parkinsonová choroba': 'P25',
    'Psychiatrické ochorenia ': 'P26',
    'Depresia': 'P27',
    'Ochorenia krčnej chrbtice': 'P28',
    'Bolesti chrbta': 'P29',
    'Reumatologické ochorenia': 'P30',
    'Nádorové ochorenie': 'P31',
    'Prekonané operácie': 'P32',
    'Prekonané úrazy': 'P33',
    'Alergie': 'P34',
    'Aké vyšetrenia ste doteraz absolvovali kvôli stratám vedomia?': 'Q',
    'Kardiologické vyšetrenia': 'Q1',
    'Záťažový test (bicyklová ergometria)': 'Q2',
    'Koronografické vyšetrenie': 'Q3',
    'HUT test': 'Q4',
    'Pažerákovú stimuláciu': 'Q5',
    'Invazívne vyšetrenie arytmií (EFV)': 'Q6',
    'Nukleárne vyšetrenie srdca (SPECT)': 'Q7',
    'CT srdca': 'Q8',
    'MRI srdca': 'Q9',
    'Neurologické vyšetrenia': 'Q10',
    'USG mozgových ciev': 'Q11',
    'CT alebo MRI mozgu': 'Q12',
    'RTG, CT alebo MRI krčnej chrbtice': 'Q13',
    'Elektromyografia (EMG)': 'Q14',
    'Psychiatrické vyšetrenie': 'Q15',
    'Endokrinologické vyšetrenie': 'Q16',
    'Odber krvi': 'Q17',
    'Iné3': 'Q18',
    'Boli ste v poslednom období očkovaní (cca za posledných 10-15 rokov)?': 'R',
    'Proti HPV (rakovina krčka maternice)': 'R1',
    'Proti chrípke': 'R2',
    'Iné':'R3',
    'Aké lieky užívate': 'S',
}, inplace=True)
# Funkcia spracuje všetky bloky typu A, B, C... kde stĺpce majú tvar A1, A2 atď.
def process_blocks_final_fixed_v2(df):
    df_transformed = df.copy()
    block_columns = [col for col in df.columns if re.match(r'^[A-Z]\d+$', col)]
    # Vytvorenie zoznamu stĺpcov pre každý blok (A: A1–A10, B: B1–B5 ...)
    blocks = {}
    for col in block_columns:
        block = re.match(r'^([A-Z])', col).group(1)
        blocks.setdefault(block, []).append(col)
    # Odstránenie niektorých stĺpcov z bloku A, ktoré nie sú binárne (napr. tlak, pulz)
    if 'A' in blocks:
        blocks['A'] = [col for col in blocks['A'] if col not in ['A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9']]
        if not blocks['A']:
            blocks.pop('A')
            
    for block, cols in blocks.items():
        if block in df.columns and len(cols) > 0:
            subset = df_transformed[cols].copy()
            # Ak aspoň jedna hodnota je 1, blok = 1 (a ostatné = 0 ak nie sú 1)
            has_one = subset.eq(1).any(axis=1)
            all_negative_one = subset.eq(-1).all(axis=1)
            df_transformed.loc[has_one, cols] = subset.loc[has_one].where(subset.loc[has_one] == 1, 0)
            df_transformed.loc[has_one, block] = 1
            # Ak sú všetky hodnoty -1, celý blok nastavíme na -1
            df_transformed.loc[all_negative_one, cols] = -1
            df_transformed.loc[all_negative_one, block] = -1
    return df_transformed

df_transformed = process_blocks_final_fixed_v2(df_transposed)

### 9. Výpočet veku v čase udalosti

 **Popis kroku:**  
Pre stĺpce `C1`, `C3`, `C4` sa vypočíta vek pacienta v čase udalosti.

**Imputácia neznámych údajov:**  
Ak vek nie je známy, nahrádza sa pomocou priemerného rozdielu oproti základnému veku pacienta.

**Použité funkcie:**
```python
def calculate_age(...)
def calculate_and_replace_age(...)




In [9]:
# Aplikácia výpočtu veku pre viaceré stĺpce naraz
def calculate_and_replace_age(df, birthdate_col, event_cols):
    for col in event_cols:
        if col in df.columns:
            df[col] = df.apply(
                lambda row: calculate_age(row[birthdate_col], row[col]), axis=1
            )
    return df
# Pomocná funkcia na výpočet veku pri konkrétnej udalosti
def calculate_age(birthdate, event_date):
    try:
        birthdate = str(birthdate)
        birth_year = int(birthdate[:4])
        birth_month = int(birthdate[5:7])
        birth_day = int(birthdate[8:10])
        event_date = str(event_date)
        event_year = int(event_date[:4])
        event_month = int(event_date[5:7])
        event_day = int(event_date[8:10])
        age = event_year - birth_year
        if (event_month, event_day) < (birth_month, birth_day):
            age -= 1
        return max(age, -1)
    except (ValueError, TypeError):
        return -1
# Zoznam stĺpcov s udalosťami, ku ktorým chceme poznať vek pacienta
event_date_columns = ["C1", "C3", "C4"]
df_transformed = calculate_and_replace_age(df_transformed, "Datum narodenia", event_date_columns)
# Funkcia na výpočet priemerného rozdielu medzi aktuálnym vekom a vekom pri udalosti
def calculate_mean_age_difference_single(df, event_col, age_col):
    valid_rows = df[(df[event_col] != -1) & (df[age_col] != -1)]
    if not valid_rows.empty:
        return (valid_rows[age_col] - valid_rows[event_col]).mean()
    return 0
# Výpočet priemerných rozdielov pre každý stĺpec
mean_age_difference_C1 = calculate_mean_age_difference_single(df_transformed, "C1", "Vek")
mean_age_difference_C3 = calculate_mean_age_difference_single(df_transformed, "C3", "Vek")
mean_age_difference_C4 = calculate_mean_age_difference_single(df_transformed, "C4", "Vek")
# Doplnenie chýbajúcich hodnôt pomocou vypočítaného priemeru
df_transformed["C1"] = df_transformed.apply(
    lambda row: mean_age_difference_C1 if row["C1"] == -1 else row["C1"],
    axis=1
)
df_transformed["C3"] = df_transformed.apply(
    lambda row: row["Vek"] - mean_age_difference_C3 if row["C3"] == -1 else row["C3"],
    axis=1
)
df_transformed["C4"] = df_transformed.apply(
    lambda row: row["Vek"] - mean_age_difference_C4 if row["C4"] == -1 else row["C4"],
    axis=1
)

### 10. Zaokrúhľovanie a export dát

**Popis kroku:**  
Hodnoty veku a dátové polia sa zaokrúhľujú a konvertujú do vhodného formátu.

**Výstup:**  
Finálny dataset sa ukladá do CSV súboru pre ďalšiu analýzu a modelovanie.


In [10]:
df_transformed["C1"] = df_transformed["C1"].round().astype(float)
df_transformed["C3"] = df_transformed["C3"].round().astype(float)
df_transformed["C4"] = df_transformed["C4"].round().astype(float)
df_transformed["Vek"] = df_transformed["Vek"].round().astype(float)
df_transformed["P1"] = df_transformed["P1"].round().astype(int)
df_transformed["P2"] = df_transformed["P2"].round().astype(int)
df_transformed["O3"] = df_transformed["O3"].round().astype(int)
df_transformed["O2"] = df_transformed["O2"].round().astype(int)
df_transformed.to_csv(r"C:\Users\admin\anaconda\envs\new_env\data_full1.csv", index=False)